In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from livelossplot import PlotLossesKeras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
import warnings
warnings.filterwarnings('ignore')

In [46]:
# set random seed globally
my_seed = 21
from numpy.random import seed
seed(my_seed)
import tensorflow as tf
tf.random.set_seed(my_seed) 
# tensorflow version
print(tf.__version__)

2.10.0


MAGIC Gamma Telescope Data Set

In [47]:
# Import dataframe from https://archive.ics.uci.edu/ml/datasets/magic+gamma+telescope

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data',
                 names = ['fLength','fWidth','fSize','fConc','fConc1','fAsym','fM3Long','fM3Trans','fAlpha','fDist','class'])
df

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g
...,...,...,...,...,...,...,...,...,...,...,...
19015,21.3846,10.9170,2.6161,0.5857,0.3934,15.2618,11.5245,2.8766,2.4229,106.8258,h
19016,28.9452,6.7020,2.2672,0.5351,0.2784,37.0816,13.1853,-2.9632,86.7975,247.4560,h
19017,75.4455,47.5305,3.4483,0.1417,0.0549,-9.3561,41.0562,-9.4662,30.2987,256.5166,h
19018,120.5135,76.9018,3.9939,0.0944,0.0683,5.8043,-93.5224,-63.8389,84.6874,408.3166,h


In [48]:
def buildDataSet (dataframe, test_data_fraction):
    # encoding the target column
    le = LabelEncoder()
    label = le.fit_transform(df['class'])
    label

    encoded_df = df.copy()
    encoded_df.drop("class", axis=1, inplace=True)
    encoded_df["class"] = label

    # Set the total number of classes
    nb_classes = len(encoded_df['class'].unique())

    # Creating target and features
    X = encoded_df.drop(['class'], axis=1)
    y = encoded_df['class']

    # scale the variables
    sc = StandardScaler() 
    X_scaled = sc.fit_transform(X)

    # Split into train and test set and normalize data
    X_train, X_test, y_train, y_test = train_test_split(X_scaled,y, test_size = test_data_fraction,stratify=y) #, random_state = 0)


    return encoded_df, nb_classes, X_train, X_test, y_train, y_test


class CustomWeight(tf.keras.initializers.Initializer):
    def __init__(self):
        print('gg')
    def constant_weight(self,num):
        return tf.keras.initializers.Constant(num)
    
    def normal_weight(self,mean,stddev):
        return tf.keras.initializers.RandomNormal(mean=0., stddev=1.)


def buildSequentialModel(hidden_layers_activation, first_layer_node_count, dropout_fraction, nb_classes,weight_class, wt_init, bias_init):
    # Build a Sequential Model.
    model = Sequential()
    # model.add(Flatten(input_shape=(28, 28)))
    
    
    model.add(Dense(first_layer_node_count, kernel_initializer=weight_class.constant_weight(wt_init), activation=hidden_layers_activation,bias_initializer=weight_class.constant_weight(bias_init)))
    model.add(Dropout(dropout_fraction))
    model.add(Dense(first_layer_node_count*0.8, kernel_initializer=weight_class.constant_weight(wt_init), activation=hidden_layers_activation,bias_initializer=weight_class.constant_weight(bias_init)))
    model.add(Dropout(dropout_fraction))
    model.add(Dense(first_layer_node_count*0.6, kernel_initializer=weight_class.constant_weight(2.2), activation=hidden_layers_activation,bias_initializer=weight_class.constant_weight(bias_init)))
    model.add(Dropout(dropout_fraction))
    
    # Output Layer
    model.add(Dense(nb_classes, activation=hidden_layers_activation, kernel_initializer=weight_class.constant_weight(2.2), bias_initializer=weight_class.constant_weight(bias_init)))
   
    return model


''' def reset_weights(model):
    for layer in model.layers:
        if isinstance(layer, tf.keras.Model): #if you're using a model as a layer
            reset_weights(layer) #apply function recursively
            continue

        #where are the initializers?
        if hasattr(layer, 'cell'):
            init_container = layer.cell
        else:
            init_container = layer

        for key, initializer in init_container.__dict__.items():
            if "initializer" not in key: #is this item an initializer?
                  continue #if no, skip it

            # find the corresponding variable, like the kernel or the bias
            if key == 'recurrent_initializer': #special case check
                var = getattr(init_container, 'recurrent_kernel')
            else:
                var = getattr(init_container, key.replace("_initializer", ""))

            var.assign(initializer(var.shape, var.dtype))
            #use the initializer '''


def Study_Activation_Functions(hidden_layers_activation, first_layer_node_count, dropout_fraction, classes, X_train, X_test, y_train, y_test,weight_class, wt_init, bias_init):
    
    # instantiate model
    model = buildSequentialModel(hidden_layers_activation, first_layer_node_count, dropout_fraction, classes,weight_class, wt_init, bias_init)
    # compile model
    #model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

    model.fit(X_train, y_train, batch_size=100, epochs=50, verbose=1, validation_data=(X_test, y_test))

    train_score = model.evaluate(X_train, y_train, verbose=0)[1]
    test_score = model.evaluate(X_test, y_test, verbose=0)[1] 

    predicted_classes = np.argmax(model.predict(X_test), axis=-1)
    correct_indices = np.nonzero(predicted_classes == y_test.values)[0]
    incorrect_indices = np.nonzero(predicted_classes != y_test.values)[0]

    return train_score, test_score

In [49]:
# build dataset on our dataframe
encoded_dataframe, classes, Train_FEATURE_matrix, Test_FEATURE_matrix, Train_TARGET_matrix, Test_TARGET_matrix = buildDataSet (df, 0.2)

print(f"shape of dataframe = {encoded_dataframe.shape}")
print(f"classes in dataset = {classes}")
print(f"X_train.shape = {Train_FEATURE_matrix.shape}")
print(f"X_test.shape = {Test_FEATURE_matrix.shape}")
print(f"y_train.shape = {Train_TARGET_matrix.shape}")
print(f"y_test.shape = {Test_TARGET_matrix.shape}")
print("")
print("encoded_dataframe shown below:")
encoded_dataframe

shape of dataframe = (19020, 11)
classes in dataset = 2
X_train.shape = (15216, 10)
X_test.shape = (3804, 10)
y_train.shape = (15216,)
y_test.shape = (3804,)

encoded_dataframe shown below:


,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,0
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,0
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,0
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,0
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,0
...,...,...,...,...,...,...,...,...,...,...,...
19015,21.3846,10.9170,2.6161,0.5857,0.3934,15.2618,11.5245,2.8766,2.4229,106.8258,1
19016,28.9452,6.7020,2.2672,0.5351,0.2784,37.0816,13.1853,-2.9632,86.7975,247.4560,1
19017,75.4455,47.5305,3.4483,0.1417,0.0549,-9.3561,41.0562,-9.4662,30.2987,256.5166,1
19018,120.5135,76.9018,3.9939,0.0944,0.0683,5.8043,-93.5224,-63.8389,84.6874,408.3166,1


Conventional Activation Functions to test

- Logistic regression hypothesis (Sigmoid)
- Hyperbolic Tangent (tanh) # rescaled sigmoid to (-1, +1)
- Rectified Linear Unit (ReLU)
- Gaussian Error Linear Unit (GELU) # smoothed ReLU
- Normalized Exponential Function (Softmax)

In [50]:
iterations = 1 # number of iterations that we plan to perform for each pair of hidden_layer_af - output_layer_af
dataframes = [pd.DataFrame() for _ in range(0, iterations)] # declare a list of empty dataframes to store the results of the iterations

for df_index in range(0, iterations):
    dataframes[df_index] = pd.DataFrame(columns = ['data_Dimension', 'Classes', 'Hidden_AF', 'TrainTest_Split', 'First_Hidden_Layer_Tensor_Count', 'Dropouts', 'Train_Accuracy', 'Test_Accuracy'])

# results_df = pd.DataFrame(columns = ['data_Dimension', 'Classes', 'Hidden_AF', 'Output_AF', 'TrainTest_Split', 'First_Hidden_Layer_Tensor_Count', 'Dropouts', 'Train_Accuracy', 'Test_Accuracy'])

Hidden_AF_list = ['relu', 'gelu', 'selu', 'sigmoid', 'tanh', 'softmax']
# Output_AF_list = ['gelu', 'sigmoid']
First_Hidden_Layer_Tensor_Count = 100
TrainTest_Split = 0.25
Dropouts = 0.3
wt_init=2.2
bias_init=0

for df_index in range(0, iterations):
    i = 0
    for haf in range(0,len(Hidden_AF_list)):
        encoded_dataframe, classes, Train_FEATURE_matrix, Test_FEATURE_matrix, Train_TARGET_matrix, Test_TARGET_matrix = buildDataSet (df, TrainTest_Split)
        train_accuracy_score, test_accuracy_score = Study_Activation_Functions(Hidden_AF_list[haf], First_Hidden_Layer_Tensor_Count, Dropouts, classes, Train_FEATURE_matrix, Test_FEATURE_matrix, Train_TARGET_matrix, Test_TARGET_matrix,CustomWeight(), wt_init, bias_init)
        dataframes[df_index].loc[i] = [encoded_dataframe.shape, classes, Hidden_AF_list[haf], TrainTest_Split, First_Hidden_Layer_Tensor_Count, Dropouts, np.round(train_accuracy_score,3), np.round(test_accuracy_score,3)]
        i = i + 1
    # model_.reset_states()
    # reset_weights(model_)    
     
    # print(f"dataframe {df_index + 1}:")
    # print(dataframes[df_index])
    # print("")

gg
149/149 [==============================] - 2s 13ms/step
gg
149/149 [==============================] - 2s 9ms/step
gg
149/149 [==============================] - 2s 9ms/step
gg
149/149 [==============================] - 1s 8ms/step
gg
149/149 [==============================] - 2s 11ms/step
gg
149/149 [==============================] - 1s 8ms/step


In [51]:
results_df = dataframes[0]
results_df = results_df.rename(columns={"Train_Accuracy": "Robust_Train_Accuracy", "Test_Accuracy": "Robust_Test_Accuracy"})
for df_index in range(1, iterations):
    results_df['Robust_Train_Accuracy'] = results_df['Robust_Train_Accuracy'] + dataframes[df_index]['Train_Accuracy']
    results_df['Robust_Test_Accuracy'] = results_df['Robust_Test_Accuracy'] + dataframes[df_index]['Test_Accuracy']
results_df['Robust_Train_Accuracy'] = round((results_df['Robust_Train_Accuracy'] / iterations),4)
results_df['Robust_Test_Accuracy'] = round((results_df['Robust_Test_Accuracy'] / iterations),4)

results_df.to_csv("results_df.csv")
results_df.sort_values(by = 'Robust_Test_Accuracy', ascending=False)

,data_Dimension,Classes,Hidden_AF,TrainTest_Split,First_Hidden_Layer_Tensor_Count,Dropouts,Robust_Train_Accuracy,Robust_Test_Accuracy
0,"(19020, 11)",2,relu,0.25,100,0.3,0.648,0.648
2,"(19020, 11)",2,selu,0.25,100,0.3,0.648,0.648
3,"(19020, 11)",2,sigmoid,0.25,100,0.3,0.648,0.648
4,"(19020, 11)",2,tanh,0.25,100,0.3,0.648,0.648
5,"(19020, 11)",2,softmax,0.25,100,0.3,0.648,0.648
1,"(19020, 11)",2,gelu,0.25,100,0.3,0.516,0.509
